In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import treecorr
import healpy as hp
import os
import time
from sklearn.preprocessing import MinMaxScaler

start_time = time.time()

import sys
sys.path.append('src')

In [2]:
GETBACK=sys.path

In [3]:
current_path = os.getcwd()  # Obtiene el directorio actual
parent_path = os.path.dirname(current_path)  # Sube un nivel
os.chdir(parent_path)  # Cambia al directorio padre
sys.path = os.getcwd()

In [4]:
GETBACK2=sys.path

In [5]:
cat = pd.read_parquet("property_maps_desy3.parquet.gzip_rlkey=raykn68b2908fkvhhw9cpz2g8")

In [6]:
sys.path=GETBACK

In [7]:
cat=cat[(cat['FOREGROUND_MASK']==0) & (cat['BADREGIONS_MASK']<=1) & (cat['FRACDET']>0.8)]
catwo=cat.copy(deep=True)
cat=[]
catwo = catwo.drop(['HPIX_4096','FOREGROUND_MASK','BADREGIONS_MASK','EBV_SFD98'], axis=1)

In [8]:
from importlib import reload
import OR_weights
reload(OR_weights)
from OR_weights import *
import somoclu

som_dim = 100
JoyDiv=100
dif=0
JD=10
SJ=3

topology = 'hexagonal'

som = somoclu.Somoclu(som_dim, som_dim+dif, gridtype=topology, initialization='pca',
                      compactsupport=False)

s2s = som2stats(som)

In [9]:
MainArray=[som_dim,JoyDiv,dif,JD,SJ]

In [10]:
scaler = MinMaxScaler()
catwo = pd.DataFrame(scaler.fit_transform(catwo), columns=catwo.columns)

In [11]:
DataArray=catwo.to_numpy()

In [12]:
%time s2s.train_som(DataArray)

CPU times: user 2d 1h 9min 45s, sys: 59.7 s, total: 2d 1h 10min 44s
Wall time: 3h 19min 12s


In [13]:
catwo = catwo.iloc[0:0]
del DataArray

In [14]:
sys.path=GETBACK2
os.chdir("Modulos")

In [15]:
with open("Modulo_SOM_BMUS.txt", "w") as file:
    for a, b in zip(s2s.training_bmus[:,0], s2s.training_bmus[:,1]):
        file.write(f"{a}\t{b}\n")

In [16]:
arr_1d = s2s.som.codebook.ravel()

In [ ]:
np.savetxt("Modulo_SOM_RESULTS.txt", arr_1d)
del arr_1d

In [ ]:
np.savetxt("Main_Parmeters.txt", MainArray)
del MainArray